In [1]:
# Main file
# Use case: Pilot test, "integration test"

# === STEP 0: Imports
print("# === STEP 0: Imports: Starting ...")
from modules.dataset_generator.helpers.configuration_loader import ConfigurationLoader as DSConfigLoader
from modules.dataset_generator.factories.data_io_factory import DataIOFactory
from modules.dataset_generator.factories.feature_processor_factory import FeatureProcessorFactory
from modules.dataset_generator.factories.join_factory import JoinFactory
from modules.dataset_generator.factories.strategy_factory import StrategyFactory
from modules.dataset_generator.dataset_generator import DatasetGenerator
from modules.processor.helpers.configuration_loader import ConfigurationLoader as PConfigLoader
from modules.processor.factories.split_strategy_factory import SplitStrategyFactory
from modules.processor.processor import Processor
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
print("# === STEP 0: Imports: Complete")

# === STEP 0: Imports: Starting ...


2024-12-27 10:46:09.542359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# === STEP 0: Imports: Complete


In [2]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...")
## === DATASET GEN
yaml_path = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v0.yaml'
ds_configuration_loader = DSConfigLoader()
data_factory, feature_processor_factory, join_factory, strategy_factory = DataIOFactory(), FeatureProcessorFactory(), JoinFactory(), StrategyFactory()
## === PROCESSOR
p_configuration_loader = PConfigLoader()
split_strategy_factory = SplitStrategyFactory()
## === MODEL MANAGER
trainer = Trainer()
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete")

# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...
# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete


In [3]:
# === STEP 2: DATASET GEN
print("# === STEP 2: DATASET GEN: Starting ...")
dataset_generator = DatasetGenerator(yaml_path, ds_configuration_loader, data_factory, feature_processor_factory, join_factory, strategy_factory)

# === STEP 2: DATASET GEN: Starting ...


In [5]:
processed_dataset = dataset_generator.generate()
print("# === STEP 2: DATASET GEN: Complete")

/Users/joaquinuriarte/Documents/GitHub/sports-betting/modules/dataset_generator/helpers/dataset_loader.py:58: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column_name] = pd.to_datetime(


Failed to cast 99 rows in column PTS_home of /Users/joaquinuriarte/Desktop/dataset/games.csv. Dropping these rows.
Invalid data: 19175   NaN
19176   NaN
19177   NaN
19178   NaN
19179   NaN
         ..
19269   NaN
19270   NaN
19271   NaN
19278   NaN
19279   NaN
Name: PTS_home, Length: 99, dtype: float64


/Users/joaquinuriarte/Documents/GitHub/sports-betting/modules/dataset_generator/helpers/dataset_loader.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = pd.to_numeric(


Invalid format in column MIN in /Users/joaquinuriarte/Desktop/dataset/games_details.csv. Dropping 134549 rows.
Invalid data: 13        nan
51        nan
62        nan
63        nan
64        nan
         ... 
668623     19
668624     23
668625     15
668626     19
668627     27
Name: MIN, Length: 134549, dtype: object


/Users/joaquinuriarte/Documents/GitHub/sports-betting/modules/dataset_generator/helpers/dataset_loader.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = pd.to_numeric(
INFO:root:Skipping game 20300001 due to insufficient player data.
INFO:root:Skipping game 20300002 due to insufficient player data.
INFO:root:Skipping game 20300003 due to insufficient player data.
INFO:root:Skipping game 20300004 due to insufficient player data.
INFO:root:Skipping game 20300005 due to insufficient player data.
INFO:root:Skipping game 20300006 due to insufficient player data.
INFO:root:Skipping game 20300007 due to insufficient player data.
INFO:root:Skipping game 20300008 due to insufficient player data.
INFO:root:Skipping game 20300009 due to

# === STEP 2: DATASET GEN: Complete


In [4]:
# === STEP 3: PROCESSOR
print("# === STEP 3: PROCESSOR: Starting ...")
processor = Processor(yaml_path, p_configuration_loader, processed_dataset, split_strategy_factory)

# === STEP 3: PROCESSOR: Starting ...


In [28]:
train_dataset, validation_dataset = processor.generate(val_dataset_flag=True)
print("# === STEP 3: PROCESSOR: Complete")

KeyError: '11400001'

In [ ]:
# === STEP 4: MODEL MANAGER
print("# === STEP 4: MODEL MANAGER: Starting ...")
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)

In [ ]:
models_and_config = model_manager.create_models([yaml_path]) # No use for the ModelConfig, could spare returning it

In [ ]:
for item in models_and_config:
    model_manager.train([item[0]], [tuple(train_dataset, validation_dataset)], save_after_training=True)
    # train needs to return the validation prediction. Otherwise we can call predict on validation as shown below
    validation_predictions = model_manager.predict([item[0]], [validation_dataset.examples])
    print(f"Validation_predictions: {validation_predictions}")
    print(f"Validation_dataset: {validation_dataset}")
print("# === STEP 4: MODEL MANAGER: Complete")